In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MaxAbsScaler

target_names = [
    "Big Tech & Startups",
    "Science & Futuristic Technology",
    "Programming, Design & Data Science",
]

In [ ]:
def load_datasets(prefix):
    train_df = pd.read_pickle(f'data/{prefix}_training.pkl')
    validation_df = pd.read_pickle(f'data/{prefix}_validation.pkl')
    test_df = pd.read_pickle(f'data/{prefix}_test.pkl')

    return train_df, validation_df, test_df

In [ ]:
# Load the datasets
articles_train_df, articles_validation_df, articles_test_df = load_datasets("articles")
sentences_train_df, sentences_validation_df, sentences_test_df = load_datasets("sentences")

# Prepare the data and labels for articles
articles_training_data = articles_train_df['text'].tolist()
articles_training_labels = articles_train_df['category_code'].tolist()
articles_test_data = articles_test_df['text'].tolist()
articles_test_labels = articles_test_df['category_code'].tolist()
articles_validation_data = articles_validation_df['text'].tolist()
articles_validation_labels = articles_validation_df['category_code'].tolist()


# Prepare the data and labels for sentences
sentences_training_data = sentences_train_df['text'].tolist()
sentences_training_labels = sentences_train_df['category_code'].tolist()
sentences_test_data = sentences_test_df['text'].tolist()
sentences_test_labels = sentences_test_df['category_code'].tolist()
sentences_validation_data = sentences_validation_df['text'].tolist()
sentences_validation_labels = sentences_validation_df['category_code'].tolist()

In [ ]:
def train_and_evaluate_multinomial_nb(training_data, training_labels, test_data, test_labels):
    vectorizer = CountVectorizer()
    training_features = vectorizer.fit_transform(training_data)
    test_features = vectorizer.transform(test_data)

    classifier = MultinomialNB()
    classifier.fit(training_features, training_labels)
    predictions = classifier.predict(test_features)

    return classification_report(test_labels, predictions, target_names=target_names)

In [ ]:
def train_and_evaluate_logistic_regression(training_data, training_labels, test_data, test_labels):
    vectorizer = CountVectorizer()
    training_features = vectorizer.fit_transform(training_data)
    test_features = vectorizer.transform(test_data)

    pipeline = make_pipeline(MaxAbsScaler(), LogisticRegression())
    pipeline.fit(training_features, training_labels)
    predictions = pipeline.predict(test_features)

    return classification_report(test_labels, predictions, target_names=target_names)

In [ ]:
def train_and_evaluate_tfidf_logistic_regression(training_data, training_labels, test_data, test_labels):
    vectorizer = TfidfVectorizer()
    training_features = vectorizer.fit_transform(training_data)
    test_features = vectorizer.transform(test_data)

    classifier = LogisticRegression()
    classifier.fit(training_features, training_labels)
    predictions = classifier.predict(test_features)

    return classification_report(test_labels, predictions, target_names=target_names)

In [ ]:
def rename_labels(dataset):
    words = {'train','validation','test'}
    try:
        for word in words:
            dataset[word]=dataset[word].rename_column ("category_code", "label")
    except:
        pass  

In [ ]:
from transformers import AutoTokenizer

pretrained_model = "bert-base-multilingual-uncased"
tokenizer = AutoTokenizer.from_pretrained(pretrained_model)

def tokenize_function(examples):

    return tokenizer(examples["text"], padding="max_length", truncation=True)

In [ ]:
import pyarrow as pa
import datasets
from datasets import Dataset

def create_dataset(training_df, validation_df, test_df):
    dataset = datasets.DatasetDict({
        "train":Dataset(pa.Table.from_pandas(training_df)),
        "validation":Dataset(pa.Table.from_pandas(validation_df)),
        "test":Dataset(pa.Table.from_pandas(test_df))})
    rename_labels(dataset)
    return dataset
    
dataset = create_dataset(articles_train_df, articles_validation_df, articles_test_df)
tokenized_dataset = dataset.map(tokenize_function, batched=True)
print(tokenized_dataset)

In [ ]:
import evaluate

metric = evaluate.load("accuracy")

In [ ]:
import numpy as np

def compute_metrics(eval_pred):

    logits, labels = eval_pred

    predictions = np.argmax(logits, axis=-1)

    return metric.compute(predictions=predictions, references=labels)

In [ ]:
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(pretrained_model, num_labels=4)

In [ ]:
from transformers import TrainingArguments

training_args = TrainingArguments(output_dir="test_trainer", evaluation_strategy="epoch")

In [ ]:
from transformers import Trainer

trainer = Trainer(

    model=model,

    args=training_args,

    train_dataset=tokenized_dataset["train"],

    eval_dataset=tokenized_dataset["validation"],

    compute_metrics=compute_metrics,

)

In [ ]:
trainer.train()
trainer.save_model(pretrained_model+"_tldr")

In [ ]:
from transformers import pipeline

text_classifier = pipeline(

    task="text-classification", model=f'./{pretrained_model}_tldr',
    tokenizer = tokenizer,

)

In [ ]:
verbose = True
full = True

test_subset = tokenized_dataset["test"]

if not full:
  N=20
  test_subset=test_subset.shuffle(seed=77).select(range(N))

y_pred = []
y_true = []

for num in range(len(test_subset)):

  if num % 10 == 0:
    print ("Ho classificato "+str(num)+" su "+str(len(test_subset))+" esempi")

  preds = text_classifier(test_subset[num]['text'][0:512])
  y_pred.append(int(preds[0]['label'].split('_')[-1]))
  y_true.append(int(test_subset[num]['label']))
  if verbose:  
    if int(y_true[-1]) != int(y_pred[-1]):
      print ('classificazione: ', target_names[y_pred[-1]])
      print ('ground truth: ', target_names[y_true[-1]])
      print (test_subset[num]['text'])
  

In [ ]:
from sklearn.metrics import classification_report
print(classification_report(y_true, y_pred, target_names=target_names))

In [ ]:
print("MultinomialNB for articles:")
print(train_and_evaluate_multinomial_nb(articles_training_data, articles_training_labels, articles_test_data, articles_test_labels))

print("Logistic Regression for articles:")
print(train_and_evaluate_logistic_regression(articles_training_data, articles_training_labels, articles_test_data, articles_test_labels))

print("TF-IDF Logistic Regression for articles:")
print(train_and_evaluate_tfidf_logistic_regression(articles_training_data, articles_training_labels, articles_test_data, articles_test_labels))

In [ ]:
print("MultinomialNB for sentences:")
print(train_and_evaluate_multinomial_nb(sentences_training_data, sentences_training_labels, sentences_test_data, sentences_test_labels))

print("Logistic Regression for sentences:")
print(train_and_evaluate_logistic_regression(sentences_training_data, sentences_training_labels, sentences_test_data, sentences_test_labels))

print("TF-IDF Logistic Regression for sentences:")
print(train_and_evaluate_tfidf_logistic_regression(sentences_training_data, sentences_training_labels, sentences_test_data, sentences_test_labels))